<h2>导入包</h2>

In [7]:
import logging
from datetime import datetime
import numpy as np
import pandas as pd
from pandas import Series
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from lightgbm import Dataset


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

<h2>读取数据</h2>

In [8]:
x_train = pd.read_csv("../data/A榜-训练集_分布式光伏发电预测_气象变量数据.csv", encoding="gbk")
y_train = pd.read_csv("../data/A榜-训练集_分布式光伏发电预测_实际功率数据.csv", encoding="gbk")
info_train = pd.read_csv("../data/A榜-训练集_分布式光伏发电预测_基本信息.csv", encoding="gbk")
x_test = pd.read_csv("../data/A榜-测试集_分布式光伏发电预测_气象变量数据.csv", encoding="gbk")
y_test = pd.read_csv("../data/submit_example.csv", encoding="utf-8")
info_test = pd.read_csv("../data/A榜-测试集_分布式光伏发电预测_基本信息.csv", encoding="gbk")

In [9]:
x_train = pd.merge(x_train, info_train[["光伏用户编号", "装机容量(kW)", "经度", "纬度"]], how="left", on="光伏用户编号")
x_train["时间"] = pd.to_datetime(x_train["时间"])
x_test = pd.merge(x_test, info_test[["光伏用户编号", "装机容量(kW)", "经度", "纬度"]], how="left", on="光伏用户编号")
x_test["时间"] = pd.to_datetime(x_test["时间"])

In [10]:
y_train = y_train.set_index(["光伏用户编号", "综合倍率", "时间"]).stack().reset_index().rename(columns={0:"target"})
y_train["level_3"] = y_train["level_3"].apply(lambda x: int(x[1:]))
y_train["时间"] = pd.to_datetime(y_train["时间"])
y_train["时间"] = y_train["时间"] + (y_train["level_3"] - 1) * 15 * pd.Timedelta(1, unit="minutes")
y_train = y_train.drop(columns=["level_3"])

y_test = y_test.set_index(["光伏用户编号", "综合倍率", "时间"]).stack().reset_index().rename(columns={0:"target"})
y_test["level_3"] = y_test["level_3"].apply(lambda x: int(x[1:]))
y_test["时间"] = pd.to_datetime(y_test["时间"])
y_test["时间"] = y_test["时间"] + (y_test["level_3"] - 1) * 15 * pd.Timedelta(1, unit="minutes")
y_test = y_test.drop(columns=["level_3"])

In [11]:
df_train = pd.merge(x_train, y_train, on=["光伏用户编号", "时间"], how="left")
df_test = pd.merge(x_test, y_test, on=["光伏用户编号", "时间"], how="left")
df = pd.concat([df_train, df_test], axis=0)

<h2>特征工程</h2>

<h3>时间特征</h3>

In [12]:
df["年"] = df["时间"].dt.year
df["月"] = df["时间"].dt.month
df["日"] = df["时间"].dt.day
df["分"] = df["时间"].dt.minute // 15 + df["时间"].dt.hour * 4

In [13]:
df["分_"] = df["分"].copy()
df = pd.get_dummies(df, columns=["分_"], prefix_sep="")

<h3>光伏用户编号</h3>

In [14]:
df["光伏用户编号_"] = df["光伏用户编号"].copy()
df = pd.get_dummies(df, columns=["光伏用户编号_"], prefix_sep="")

<h3>历史值特征</h3>

In [15]:
dfs = []
for site, df_site in df.groupby("光伏用户编号"):
    df_site["辐照强度（J/m2） - 1"] = df_site["辐照强度（J/m2）"].shift(1) - df_site["辐照强度（J/m2）"]
#     df_site["辐照强度（J/m2） - 2"] = df_site["辐照强度（J/m2）"].shift(2) - df_site["辐照强度（J/m2）"]
    dfs.append(df_site)
df = pd.concat(dfs, axis=0)

<h3>划分测试集</h3>

In [16]:
df_train = df[df["时间"] <= df_train["时间"].max()]
df_test = df[df["时间"] >= df_test["时间"].min()]

<h2>训练模型</h2>

<h3>评测指标</h3>

In [17]:
def score(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return 1 / (1 + rmse)

<h3>lightgbm模型</h3>

In [18]:
params_lgb = {
    'learning_rate': 0.02,
    'boosting_type': 'gbdt',
    'objective': 'mse',
    'metric': 'mse',
    'num_leaves': 64,
    'verbose': -1,
    'seed': 42,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
}

<h3>交叉验证</h3>

In [19]:
model_lgb = []
kfold = KFold(n_splits=5, random_state=42, shuffle=True)

x = df_train.drop(columns=["光伏用户编号", "时间"]).astype(np.float32)
y = x.pop("target")
mse = 0
for fold, (train_index, val_index) in enumerate(kfold.split(x, y)):
    logging.info(f'############ fold: {fold} ###########')
    x_train, x_val, y_train, y_val = x.iloc[train_index], x.iloc[val_index], y.iloc[train_index], y.iloc[val_index]
    trainset = Dataset(x_train, y_train)
    valset = Dataset(x_val, y_val)
    model = lgb.train(params_lgb, trainset, valid_sets=[trainset, valset], num_boost_round=3000, callbacks=[lgb.early_stopping(100), lgb.log_evaluation(1000)])
    model.save_model("../models/lgb_%d.txt" % fold)
    model_lgb.append(model)
    val_pred = Series(model.predict(x_val, num_iteration=model.best_iteration), index=y_val.index).fillna(0)
    mse += mean_squared_error(y_val.fillna(0), val_pred)
rmse = np.sqrt(mse)
score = 1 / (1 + rmse)
logging.info(f"--------------本地分数 {score}--------------")

2024-03-10 12:24:57,173 : INFO : ############ fold: 0 ###########


Training until validation scores don't improve for 100 rounds
[1000]	training's l2: 0.0213111	valid_1's l2: 0.0263879
[2000]	training's l2: 0.0162868	valid_1's l2: 0.0237947
[3000]	training's l2: 0.013202	valid_1's l2: 0.0223841
Did not meet early stopping. Best iteration is:
[3000]	training's l2: 0.013202	valid_1's l2: 0.0223841


2024-03-10 12:25:24,572 : INFO : ############ fold: 1 ###########


Training until validation scores don't improve for 100 rounds
[1000]	training's l2: 0.0210651	valid_1's l2: 0.02709
[2000]	training's l2: 0.0158718	valid_1's l2: 0.0243811
[3000]	training's l2: 0.0129153	valid_1's l2: 0.0231651
Did not meet early stopping. Best iteration is:
[3000]	training's l2: 0.0129153	valid_1's l2: 0.0231651


2024-03-10 12:25:52,306 : INFO : ############ fold: 2 ###########


...

2024-03-10 12:26:20,013 : INFO : ############ fold: 3 ###########


Training until validation scores don't improve for 100 rounds
[1000]	training's l2: 0.021427	valid_1's l2: 0.026182
[2000]	training's l2: 0.0161623	valid_1's l2: 0.023383
[3000]	training's l2: 0.013148	valid_1's l2: 0.022133
Did not meet early stopping. Best iteration is:
[3000]	training's l2: 0.013148	valid_1's l2: 0.022133


2024-03-10 12:26:47,745 : INFO : ############ fold: 4 ###########


Training until validation scores don't improve for 100 rounds
[1000]	training's l2: 0.0215696	valid_1's l2: 0.0260855
[2000]	training's l2: 0.016372	valid_1's l2: 0.0233506
[3000]	training's l2: 0.0133712	valid_1's l2: 0.0221692
Did not meet early stopping. Best iteration is:
[3000]	training's l2: 0.0133712	valid_1's l2: 0.0221692


2024-03-10 12:27:15,338 : INFO : --------------本地分数 0.7494531551117443--------------


<h2>预测</h2>

In [20]:
x_test = df_test.drop(columns=["光伏用户编号", "时间"]).astype(np.float32)
y_test = x_test.pop("target")
y_pred = np.zeros((df_test.shape[0], ))
for model in model_lgb:
    y_pred += model.predict(x_test, num_iteration=model.best_iteration)
y_pred = y_pred / kfold.n_splits
df_test["target"] = y_pred

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [21]:
df_test = df_test[["光伏用户编号", "综合倍率", "年", "月", "日", "分", "target"]]
df_test["时间"] = df_test["年"].astype(str) + "-" + df_test["月"].astype(str) + "-" + df_test["日"].astype(str) + " 0:00"
df_test["分"] = "p" + (df_test["分"] + 1).astype(str)
df_test = df_test.drop(columns=["年", "月", "日"])

In [22]:
result = pd.pivot(df_test, index=["光伏用户编号", "综合倍率", "时间"], columns="分", values="target").reset_index()
result = result[result["综合倍率"].notnull()]
result["综合倍率"] = result["综合倍率"].astype(int)

In [23]:
result.to_csv("../data/%s.csv" % datetime.now().strftime("%Y%m%d_%H%M%S"), encoding="utf-8", index=False)